In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
# attack_feature = np.array(np.load("attack_temporal_embedding.npz", allow_pickle=True)["arr_0"])
# normal_feature = np.array(np.load("normal_temporal_embedding.npz", allow_pickle= True)["arr_0"])

In [3]:
attack_feature = np.array(np.load("attack_spatio_temporal_embeddings.npz", allow_pickle=True)["arr_0"])
normal_feature = np.array(np.load("normal_spatio_temporal_embeddings.npz", allow_pickle= True)["arr_0"])

# attack_feature = np.array(np.load("attack_spatial_embeddings.npz", allow_pickle=True)["arr_0"])
# normal_feature = np.array(np.load("normal_spatial_embeddings.npz", allow_pickle= True)["arr_0"])

# attack_feature = np.array(np.load("attack_matrix.npz", allow_pickle=True)["arr_0"])
# normal_feature = np.array(np.load("normal_matrix.npz", allow_pickle= True)["arr_0"])

# attack_feature = np.array(np.load("no_cluster_attack_spatio_temporal_embeddings.npz", allow_pickle=True)["arr_0"])
# normal_feature = np.array(np.load("no_cluster_normal_spatio_temporal_embeddings.npz", allow_pickle= True)["arr_0"])

In [4]:
attack_target_label =  np.array(np.load("newlabel_swat.npz")["arr_0"])

In [5]:
p = np.where(attack_target_label==1)

In [6]:
p[0].shape

(10942,)

In [7]:
attack_feature.shape

(89978, 51, 16)

In [8]:
def norm(x):
    return (x- torch.min(x)) / (torch.max(x)- torch.min(x))

In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader

In [10]:
normal_target_label = np.zeros(normal_feature.shape[0])
normal_feature_tensor = torch.Tensor(normal_feature)
# normal_feature_tensor = norm(normal_feature_tensor)
normal_feature_tensor = torch.unsqueeze(normal_feature_tensor, 1)
normal_label_tensor = torch.Tensor(normal_target_label)

normal_dataset = TensorDataset(normal_feature_tensor, normal_label_tensor)
normal_dataloader = DataLoader(normal_dataset, batch_size=300, shuffle=False)

attack_feature_tensor = torch.Tensor(attack_feature)
# attack_feature_tensor = norm(attack_feature_tensor)
attack_feature_tensor = torch.unsqueeze(attack_feature_tensor, 1)
attack_label_tensor = torch.Tensor(attack_target_label)

attack_dataset = TensorDataset(attack_feature_tensor, attack_label_tensor)
attack_dataloader = DataLoader(attack_dataset, batch_size=300, shuffle=False)


In [11]:
attack_pos = []
i = 0
# for i in range(0, attack_target_label.shape[0], end):
while i< attack_target_label.shape[0]:
    attack_index = []
    if attack_target_label[i] == 1:
        init = i
        attack_index.append(init)
        j = i+1
        while attack_target_label[j] == 1:
            j = j+1
        else:
            end = j-1
        attack_index.append(end)
        attack_pos.append(attack_index)
        i = end+1
    else:
        i += 1

In [12]:
attack_pos

[[346, 538],
 [609, 702],
 [980, 1060],
 [1287, 1369],
 [1447, 1490],
 [1537, 1626],
 [2278, 2474],
 [3072, 3220],
 [18133, 18183],
 [18424, 18514],
 [18685, 18744],
 [20614, 20761],
 [23164, 23220],
 [23224, 23307],
 [23396, 23544],
 [26579, 26676],
 [28586, 28730],
 [34449, 34517],
 [34578, 34704],
 [39655, 39948],
 [45561, 52745],
 [55820, 55848],
 [56008, 56246],
 [60526, 60603],
 [69531, 69655],
 [72234, 72326],
 [74291, 74315],
 [74367, 74467],
 [77932, 78043],
 [87304, 87401],
 [87479, 87539],
 [87625, 87709],
 [87720, 87783],
 [88696, 89038]]

In [13]:
attack_feature_tensor.shape

torch.Size([89978, 1, 51, 16])

In [14]:
import torch
import numpy as np

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1 and classname != 'Conv':
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("Linear") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)

In [15]:
import torch.nn as nn
import torch.nn.functional as F

In [16]:
class network(nn.Module):
    def __init__(self, z_dim=64):
        super(network, self).__init__()
        self.pool = nn.AvgPool2d(2, 2)

        self.conv1 = nn.Conv2d(1, 8, (5,3), bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, (5,3), bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(240, z_dim, bias=False)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        return self.fc1(x)

In [17]:
class autoencoder(nn.Module):
    def __init__(self, z_dim=64):
        super(autoencoder, self).__init__()
        self.z_dim = z_dim
        self.pool = nn.AvgPool2d(2,2)

        self.conv1 = nn.Conv2d(1, 8, (5,3), bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, (5,3), bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(240, z_dim, bias=False)

        self.deconv1 = nn.ConvTranspose2d(4, 8, (3,5), bias=False, padding=2)
        self.bn3 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.deconv2 = nn.ConvTranspose2d(8, 8, (3,5), bias=False, padding=2)
        self.bn4 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.deconv3 = nn.ConvTranspose2d(8, 1, (2,5), bias=False, padding=(1,2))
        
    def encode(self, x):
        x = self.conv1(x)
        x = self.pool(F.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(F.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        return self.fc1(x)
   
    def decode(self, x):
        x = x.view(x.size(0), int(self.z_dim / 16), 8, 2)
        x = F.interpolate(F.leaky_relu(x), scale_factor=2)
        x = self.deconv1(x)
        x = F.interpolate(F.leaky_relu(self.bn3(x)), scale_factor=2)
        x = self.deconv2(x)
        x = F.interpolate(F.leaky_relu(self.bn4(x)), scale_factor=2)
        x = self.deconv3(x)
        return torch.sigmoid(x)
        

    def forward(self, x):
        z = self.encode(x)
        x_hat = self.decode(z)
        return x_hat

In [25]:
from torch import optim
import torch.nn.functional as F
from barbar import Bar


class TrainerDeepSVDD:
    def __init__(self, args, data, device):
        self.args = args
        self.train_loader, self.test_loader = data
        self.device = device
    

    def pretrain(self):
        """ Pretraining the weights for the deep SVDD network using autoencoder"""
        ae = autoencoder(self.args.latent_dim).to(self.device)
        ae.apply(weights_init_normal)
        optimizer = optim.Adam(ae.parameters(), lr=self.args.lr_ae,
                               weight_decay=self.args.weight_decay_ae)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                    milestones=self.args.lr_milestones, gamma=0.1)
        
        ae.train()
        for epoch in range(self.args.num_epochs_ae):
            total_loss = 0
            for x, _ in Bar(self.train_loader):
                x = x.float().to(self.device)
                
                optimizer.zero_grad()
                x_hat = ae(x)
                reconst_loss = torch.mean(torch.sum((x_hat - x) ** 2, dim=tuple(range(1, x_hat.dim()))))
                reconst_loss.backward()
                optimizer.step()
                
                total_loss += reconst_loss.item()
            scheduler.step()
            print('Pretraining Autoencoder... Epoch: {}, Loss: {:.3f}'.format(
                   epoch, total_loss/len(self.train_loader)))
        self.save_weights_for_DeepSVDD(ae, self.train_loader) 
    

    def save_weights_for_DeepSVDD(self, model, dataloader):
        """Initialize Deep SVDD weights using the encoder weights of the pretrained autoencoder."""
        c = self.set_c(model, dataloader)
        net = network(self.args.latent_dim).to(self.device)
        state_dict = model.state_dict()
        net.load_state_dict(state_dict, strict=False)
        torch.save({'center': c.cpu().data.numpy().tolist(),
                    'net_dict': net.state_dict()}, 'pretrained_parameters.pth')
    

    def set_c(self, model, dataloader, eps=0.05):
        """Initializing the center for the hypersphere"""
        model.eval()
        z_ = []
        with torch.no_grad():
            for x, _ in dataloader:
                x = x.float().to(self.device)
                z = model.encode(x)
                z_.append(z.detach())
        z_ = torch.cat(z_)
        c = torch.mean(z_, dim=0)
        c[(abs(c) < eps) & (c < 0)] = -eps
        c[(abs(c) < eps) & (c > 0)] = eps
        return c


    def train(self):
        """Training the Deep SVDD model"""
        net = network().to(self.device)
        
        if self.args.pretrain==True:
            state_dict = torch.load('pretrained_parameters.pth')
            net.load_state_dict(state_dict['net_dict'])
            c = torch.Tensor(state_dict['center']).to(self.device)
        else:
            net.apply(weights_init_normal)
            c = torch.randn(self.args.latent_dim).to(self.device)
        
        optimizer = optim.Adam(net.parameters(), lr=self.args.lr,
                               weight_decay=self.args.weight_decay)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, 
                    milestones=self.args.lr_milestones, gamma=0.1)

        net.train()
        for epoch in range(self.args.num_epochs):
            total_loss = 0
            for x, _ in Bar(self.train_loader):
                x = x.float().to(self.device)

                optimizer.zero_grad()
                z = net(x)
                loss = torch.mean(torch.sum((z - c) ** 2, dim=1))
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            scheduler.step()
            print('Training Deep SVDD... Epoch: {}, Loss: {:.3f}'.format(
                   epoch, total_loss/len(self.train_loader)))
        self.net = net
        self.c = c

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
from sklearn.preprocessing import MinMaxScaler

def eval(net, c, dataloader, device):
    """Testing the Deep SVDD model"""

    scores = []
    labels = []
    predictions = []
    net.eval()
    print('Testing...')
    with torch.no_grad():
        for x, y in dataloader:
            x = x.float().to(device)
            z = net(x)
            score = torch.sum((z - c) ** 2, dim=1)

            scores.append(score.detach().cpu())
            labels.append(y.cpu())

    labels, scores = torch.cat(labels).numpy(), torch.cat(scores).numpy()
    scaler = MinMaxScaler(feature_range=(0, 1))
#     scaler = MinMaxScaler()
    scores = scores.reshape(-1,1)
    scaler.fit(scores)
    norm_scores = scaler.transform(scores)
    
#     for item in scores:
#         item = item - min(scores) / (max(scores)-min(scores))
#     scores = norm_list(scores)


    for item in norm_scores:
        if item>0.2:
            predictions.append(1)
        else:
            predictions.append(0)
                
    preds = np.asarray(predictions)
    
    for item in attack_pos:
        j = preds[item[0]:item[1]+1]
        for element in j:
            if element == 1:
                preds[item[0]:item[1]+1] = 1
                break
                    
    mod_preds = list(preds)
    mod_preds_array = np.asarray(mod_preds)
            
    print("Accuracy Score: {}".format(accuracy_score(labels, mod_preds)))
    print("Precision Score: {}".format(precision_score(labels, mod_preds)))
    print("Recall Score: {}".format(recall_score(labels, mod_preds)))
    print("F1 Score: {}".format(f1_score(labels, mod_preds)))
    print('ROC AUC score: {:.2f}'.format(roc_auc_score(labels, mod_preds)*100))
    return labels, scores, norm_scores, mod_preds_array

In [27]:
class Args:

    num_epochs=1
    num_epochs_ae=1
    patience=50
    lr=1e-4
    weight_decay=0.5e-6
    weight_decay_ae=0.5e-3
    lr_ae=1e-4
    lr_milestones=[50]
#     batch_size=200
    pretrain=False
    latent_dim=64
    normal_class=0
    
    
args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = (normal_dataloader, attack_dataloader)

deep_SVDD = TrainerDeepSVDD(args, data, device)

if args.pretrain:
    deep_SVDD.pretrain()

In [28]:
deep_SVDD.train()

99354/99354: [===============================>] - ETA 0.1ss
Training Deep SVDD... Epoch: 0, Loss: 10.589


In [29]:
#  from test import eval
labels, scores, norm_scores, preds = eval(deep_SVDD.net, deep_SVDD.c, data[1], device)

Testing...
Accuracy Score: 0.9586899019760385
Precision Score: 0.9971102243016375
Recall Score: 0.6622189727654908
F1 Score: 0.7958701740897358
ROC AUC score: 83.10


In [39]:
p = np.where(labels==1)
p[0].shape

(2188,)

In [24]:
q = np.where(norm_scores>0.3)
q[0].shape

(6403,)

In [71]:
max(norm_scores)

array([1.], dtype=float32)

In [57]:
scores

array([[38.281487],
       [38.259678],
       [38.2658  ],
       ...,
       [38.27997 ],
       [38.2875  ],
       [38.32311 ]], dtype=float32)